In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np 
from pprint import pprint

from model_trainer import ModelTrainer
from data_loader import DataLoader
from physics_engine import generate_data_with_simulator, N_FEATURES, OUTPUT

### N body
In the n-body domain,  such as solar systems, all n bodies exert distance- and mass-dependent gravitational forces on each other, so there were n(n−1) relations input to our model.  Across simulations, the objects’ masses varied, while all other fixed attributes were held constant

In [3]:
n_samples = 10
n_objects  = 6
relation_dim = 1
effect_dim = 100  # effect's vector size
dt = 0.001

batch_size = 1000
time_steps = 100000
n_epoch = 100

In [4]:
print('Using parameters for the QUICK RUN')
n_samples = 1
n_objects  = 6
relation_dim = 1
effect_dim = 100  # effect's vector size
dt = 0.001

batch_size = 100
time_steps = 1000
n_epoch = 10

In [27]:
data = []
for ii in range(n_samples):
    orbit = np.random.choice([True, False])
    data.append(generate_data_with_simulator(n_objects=n_objects, 
                                             orbit=orbit,
                                             time_steps=time_steps, 
                                             dt=dt))

data_loader = DataLoader(data=data,
                         scaler=None,
                         batch_size=batch_size,
                         shuffle=True)

print('len data', len(data))
print('data loader len', len(data_loader))

In [28]:
model = ModelTrainer(state_dim=N_FEATURES,
                     relation_dim=relation_dim,
                     effect_dim=effect_dim,
                     output_dim=len(OUTPUT))

In [7]:
pprint(list(model.optimizer.state_dict().keys()))
pprint(list(model.network.state_dict().keys()))

In [31]:
model.train(n_epoch=n_epoch,
            data_loader=data_loader)

In [30]:
model.save()

In [32]:
model = ModelTrainer.load()

In [33]:
model.train(n_epoch=n_epoch,
            data_loader=data_loader)

In [ ]:
plt.plot(model.losses[-5000:])

In [ ]:
for ii in range(1):
    print('Cycle', ii)
    # data = generate_data_with_simulator(n_objects=n_objects, orbit=True, time_steps=time_steps, dt=dt)
    data = generate_data_with_simulator(n_objects=n_objects, orbit=False, time_steps=time_steps, dt=dt)
    data_loader = DataLoader(data=data, batch_size=batch_size, shuffle=True)    
    model.train(n_epoch=n_epoch, data_loader=data_loader)

print('Finished')

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(model.losses[-5000:])
plt.show()
    

In [ ]:
n_objects_test = 6
test_data = generate_data_with_simulator(n_objects=n_objects_test, 
                                        orbit=not True,
                                        time_steps=100, 
                                        dt=0.001)
test_data_loader = DataLoader(data=test_data,
                             batch_size=len(test_data),
                             shuffle=False)
sim_output = model.test(data_loader=test_data_loader, dt=0.001)

In [ ]:
model.network

In [ ]:
import torch
torch.save(model.network.state_dict(), 'trained_model_state.torch')   
model.network.state_dict()

In [ ]:
model.network


In [ ]:
model.network


In [ ]:
data = []
for ii in range(1,4):
    sample = generate_data_with_simulator(n_objects=n_objects, orbit=True,
                                          time_steps=5*ii, dt=dt)
    data.append(sample)
    
data_loader = DataLoader(data=data, batch_size=None, shuffle=True)

print('data.shapes', [sample.shape for sample in data])
print('data loader len', len(data_loader))
print('data loader indexes\n', data_loader.indexes)


for X, y in data_loader:
    print(X.shape)
    print(y.shape)
    print(X)